In [3]:
import pandas as pd
import sklearn as sklearn

La biblioteca xlrd se usa para acceder a Excel como fuente de datos. Los datos originales contienen macros (xlsx), por lo tanto se requiere una bilbioteca adicional (openpyxl).

In [ ]:
!pip3 install --user xlrd
!pip3 install --user openpyxl

In [80]:
import os

df=pd.read_excel(
     os.path.join("data", "reclamos_20201221_con_clas.xlsx"),
     engine='openpyxl',
)

In [81]:
df.columns

Index(['CASO_ID', 'MERCADO_ANALISTA', 'DESCRIPCION_CIUDADANO',
       'PETICION_CIUDADANO'],
      dtype='object')

In [82]:
df['MERCADO_ANALISTA']=df['MERCADO_ANALISTA'].astype('category')

In [83]:
df.head()

,CASO_ID,MERCADO_ANALISTA,DESCRIPCION_CIUDADANO,PETICION_CIUDADANO
0,638428.0,APIA -Reclamo Seguros,RECIBI UN CORREO ELECTRONICO DONDE SE ME INDIC...,DADO QUE NINGUNA DE LAS CAUSALES ANTERIORES SE...
1,580194.0,APIA -Reclamo Seguros,Mi nombre es XX y el día 27 de Enero de este a...,Necesito que Metlife me deposite el dinero sol...
2,447321.0,APIA -Reclamo Seguros,Estimados Señores SVS\n\nCon fecha 13 de Octub...,Por esta razón quiero colocar un reclamo forma...
3,437201.0,APIA -Reclamo Seguros,En primera instancia solicité renvar mi tarjet...,Solicito devolución de lo que me cobraron por ...
4,385926.0,Reclamo Valores,reclamo en contra Scotiabank por cobros de man...,condonacion de los cobros que hace el Banco.


Al contar los datos pertencientes a cada clase, se observa desbalance de clases.

In [130]:
df.groupby('MERCADO_ANALISTA').count()

,PETICION_CIUDADANO,DESCRIPCION_CIUDADANO,data
MERCADO_ANALISTA,,,
APIA -Reclamo Seguros,662,662,662
Reclamo Valores,337,337,337


In [85]:
df.loc[0]['DESCRIPCION_CIUDADANO']

'RECIBI UN CORREO ELECTRONICO DONDE SE ME INDICA QUE  ZURICH SANTANDER SEGUROS HA DECIDIDO NO RENOVAR LA PÓLIZA Nº XX - SÚPER SEGURO ALIVIO SEGURO FAMILIAR POR UN NUEVO PERÍODO, CONFORME AL ARTÍCULO 5 DE LAS CONDICIONES GENERALES DE SU SEGURO (POL 2 99 003). \n\nSEGÚN HE REVISADO, LA POL 299 003 CORRESPONDE A UNA POLIZA DE PRESTACIONES MEDICAS, LA CUAL ESTA PROHIBIDA POR RESOLUCION N°541 DE 15/09/2010.\n\nADEMÁS REVISANDO ANTECEDENTES ENTIENDO QUE LA COMPAÑÍA NO PUEDE PONER TERMINO UNILATERAL A DICHO SEGURO A MENOS QUE SE CUMPLA ALGUNA DE LAS CAUSALES COMO “NO PAGO DE LA PRIMA; CAMBIO DE LO QUE SE ASEGURÓ; OMISIÓN O FALSEO DE INFORMACIÓN POR PARTE DEL CLIENTE; PÉRDIDA TOTAL O PARCIAL, DESTRUCCIÓN DE LOS RIESGOS O LO ASEGURADO TRAS FIRMAR EL CONTRATO, AÚN SI EL EVENTO TIENE O NO COBERTURA. CAMBIO EN LA MONEDA DE LA PÓLIZA”.\n'

In [86]:
df.loc[0]['PETICION_CIUDADANO']

'DADO QUE NINGUNA DE LAS CAUSALES ANTERIORES SE CUMPLE ES QUE VENGO A SOLICITAR A UDS INTERVENGAN EN ESTA DECISION UNILATERAL DE LA COMPAÑÍA Y SE RECHAZE LA NO RENOVACION DE MI POLIZA TOMADA HACE10 AÑOS (13/09/2006) '

In [87]:
df.loc[0]['MERCADO_ANALISTA']

'APIA -Reclamo Seguros '

In [88]:
df.loc[500]['DESCRIPCION_CIUDADANO']

'Necesito,\nSaber cual es la forma de obtener información sobre la constructora Curauma quien se dio a quiebra y no ha dado ninguna información a los acciones pequeños como yo, he tratado de comunicarme vía teléfono sin respuesta de esta entidad.'

In [89]:
df.loc[500]['PETICION_CIUDADANO']

'saber si puedo recuperar algo de mi inversion'

In [96]:
df.loc[500]['MERCADO_ANALISTA']

'Reclamo Valores'

Borramos los datos nulos

In [97]:
df=df[['MERCADO_ANALISTA','PETICION_CIUDADANO','DESCRIPCION_CIUDADANO']].dropna()

Juntamos los dos campos de texto 'PETICION_CIUDADANO' y 'DESCRIPCION_CIUDADANO'

In [131]:
df['data'] = df['DESCRIPCION_CIUDADANO'].map(str) + df['PETICION_CIUDADANO'].map(str)

In [135]:
df['data'][-20:]

979    Desde hace años mantengo inversiones en fondos...
980    Metlife Compañía de Seguros de Vida Rechaza el...
981    el problema es que depsoite dinero a esta iden...
982    CON FECHA OCTUBRE DE 2013 CONTRATE UN CREDITO ...
983    En noviembre del mes pasado, presente los ante...
984    Estimados buenos días, estoy interesado en com...
985    Muy buenos días:\n\nA continuación les envío l...
986    Buenas tardes, se ejecuto la venta de mis acci...
987    No estoy de acuerdo su carta N°  14789  del 10...
988    XX es una entidad informante por estar afecta ...
989    Junto con saludar, envió reclamo por la no dev...
990    Hace un par de años fui cliente del banco en c...
991    BUENAS TARDES ,TRATARE DE SER BREVE Y PRECISO,...
992    Estimados\nEl 26 de febrero, durante mis vacac...
993    Hace un año me entere que tenia acciones de mi...
994    Estimados por favor revisar el proceso de fusi...
995    Con fecha de febrero del 2013 me fueron protes...
996    1. Mal calculo de valor 

In [100]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import f1_score

In [101]:
n_documents = df.shape[0]
count_vectorizer = CountVectorizer(input='content', binary=True,max_df=0.25, min_df=1.01/n_documents) 
train_binary_bag_of_words = count_vectorizer.fit_transform(df['data']).todense() 


In [102]:
train_binary_bag_of_words.shape

(999, 6188)

In [124]:
labels=(df['MERCADO_ANALISTA']=='Reclamo Valores').astype(int)

In [126]:
labels.shape

(999,)